In [9]:
import datetime
import sqlite3
import pandas as pd
import numpy as np

from dateutil.relativedelta import relativedelta

Создадим базу данных для хранения информации. База данных будет размещаться локально и написана на SQLite. Таблиц в базе будет две. Главная таблица будет содержать Значения почасового потребления мощности (истинные и предсказанные) и данные о температуре. Вторая таблица будет содержать данные о выходных и рабочих днях. Единой для них колонкой (Prime and foreign) будет колонка с датой.

In [10]:
# Создадим базу данных и подключимся к ней
conn = sqlite3.connect('../energy_consumpion.sqlite')
cursor = conn.cursor()

In [11]:
# Создадим таблицы
cursor.execute('''CREATE TABLE IF NOT EXISTS consumption_table
                    (id          INTEGER PRIMARY KEY AUTOINCREMENT,
                     power_true  REAL,
                     power_pred  REAL,
                     temperature REAL,
                     datetime    TIMESTAMP)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS day_off_table
                    (id          INTEGER PRIMARY KEY AUTOINCREMENT,
                     day_off     INTEGER,
                     datetime    TIMESTAMP,
                     FOREIGN KEY (datetime) REFERENCES consumption_table (datetime))''')

Внесем в таблицу consumption_table данные о потреблении. Заранее расширим таблицу до конца года, чтобы в дальнейшем имелась возможность внести в таблицу consumption_table собранные сведения обо всех выходных днях.

In [12]:
df = pd.read_csv('../data_for_train/consumption_and_temperature_data.csv')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
df.set_index('datetime', inplace=True)
print('df.shape', df.shape)
df.head(2)

df.shape (64992, 2)


,one_hour_consumption,one_hour_temperature
datetime,,
2016-01-01 00:00:00,3962.983333,-8.313333
2016-01-01 01:00:00,3808.266667,-8.260000


Расширим таблицу до конца года, создадим новый столбец, а имеющиеся - переименуем.

In [13]:
df = pd.concat(
    [
        df,
        pd.DataFrame(columns=df.columns,
                     index=pd.date_range(start='2023-06-01 00:00:00',
                                         end='2023-12-31 23:00:00',
                                         freq='H'))
    ]
)

In [14]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'datetime',
                   'one_hour_consumption': 'power_true',
                   'one_hour_temperature': 'temperature'},
          inplace=True)
df['power_pred'] = np.nan

In [15]:
df = df[['power_true', 'power_pred', 'temperature', 'datetime']]

In [16]:
print('df.shape', df.shape)
df.head(2)

df.shape (70128, 4)


,power_true,power_pred,temperature,datetime
0,3962.983333,NaN,-8.313333,2016-01-01 00:00:00
1,3808.266667,NaN,-8.260000,2016-01-01 01:00:00


In [17]:
df.tail(2)

,power_true,power_pred,temperature,datetime
70126,NaN,NaN,NaN,2023-12-31 22:00:00
70127,NaN,NaN,NaN,2023-12-31 23:00:00


Запишем датафрейм в БД.

In [18]:
df.to_sql(name='consumption_table',
          con=conn,
          if_exists='replace',
          index=False)

70128

Прочитаем датафрейм с календарем в память.

In [19]:
calendar = pd.read_excel('../data_for_train/calendar.xlsx', index_col='day')
print('calendar.shape', calendar.shape)
calendar.head(7)

calendar.shape (2922, 1)


,day_off
day,
2016-01-01,1
2016-01-02,1
2016-01-03,1
2016-01-04,0
2016-01-05,0
2016-01-06,0
2016-01-07,1


In [20]:
calendar.reset_index(inplace=True)
calendar.rename(columns={'day': 'datetime'}, inplace=True)

In [21]:
calendar.head(2)

,datetime,day_off
0,2016-01-01,1
1,2016-01-02,1


Внесем данные в БД.

In [22]:
calendar.to_sql(name='day_off_table',
                con=conn,
                if_exists='replace',
                index=False)

2922

In [23]:
conn.close()

Данные записаны в ЛБД. Проверим её, получив из нее данные за май 2023 года.

In [45]:
conn = sqlite3.connect('../energy_consumpion.sqlite')
cursor = conn.cursor()

In [46]:
query = """
SELECT
    t.power_true,
    t.power_pred,
    t.temperature,
    t.datetime,
    (CASE WHEN day_off_table.day_off = 1 THEN 1 ELSE 0 END) AS day_off
FROM
    consumption_table AS t
    LEFT JOIN day_off_table ON DATE(t.datetime) = DATE(day_off_table.datetime)
WHERE
    t.datetime >= '2023-05-01 00:00:00' AND
    t.datetime <= '2023-05-31 23:00:00'
"""

In [47]:
sql_df = pd.read_sql(sql=query, con=conn)
conn.close()
sql_df['datetime'] = pd.to_datetime(sql_df['datetime'], format='%Y-%m-%d %H:%M:%S')
print('sql_df.shape', sql_df.shape)
sql_df.head(2)

sql_df.shape (744, 5)


,power_true,power_pred,temperature,datetime,day_off
0,3462.677124,None,6.820000,2023-05-01 00:00:00,1
1,3337.652337,None,6.333333,2023-05-01 01:00:00,1


In [48]:
sql_df.tail(2)

,power_true,power_pred,temperature,datetime,day_off
742,4337.348509,None,17.113333,2023-05-31 22:00:00,0
743,3948.607652,None,16.286666,2023-05-31 23:00:00,0


In [49]:
sql_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   power_true   744 non-null    float64       
 1   power_pred   0 non-null      object        
 2   temperature  744 non-null    float64       
 3   datetime     744 non-null    datetime64[ns]
 4   day_off      744 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 29.2+ KB
